In [1]:
# pip als Paketmanager
#! pip install -q pyscipopt
#! pip install pandas
#!pip install openpyxl

In [2]:
from pyscipopt import Model, quicksum

**Optimierungsmodell für den Kauf und Verkauf von Strom auf dem Strommarkt**

In [3]:
# Erstellen einer Modellinstanz
scip = Model()

**Indexmenge**

In [4]:
# Investition über Zeitraum j Monate
j = 1

H = [n for n in range(1, 24*30*j+1)]
R = [r for r in range(0, 25)]
# Erklärung:
# es gibt 26 Risikoabstufungen, höchstes R -> höchstes Risiko, r=0, kein Risiko
#-> kein Risiko + normaler EW, Risikoklasse 1=4% Risiko und EW ergibt sich aus top 96% der Preisprognosen für h, etc.

# Risikowahrscheinlichkeiten
prob_r = {}
for r in R:
    prob_r[r] = 1 - r/25



**Batterie-Systemspezifikationen**

In [5]:
wirkungsgrad_wechselrichter = 0.985

wirkungsgrad_laden = 0.975

round_trip_efficiency = 0.95
entlade_verlust = wirkungsgrad_laden - round_trip_efficiency

wirkungsgrad_systemeingang = wirkungsgrad_wechselrichter * wirkungsgrad_laden
wirkungsgrad_systemausgang = (1-(entlade_verlust / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter



f_e = wirkungsgrad_systemeingang # Faktor Einkauf
f_v = wirkungsgrad_systemausgang # Faktor Verkauf

# Durch investition verbesserte Wirkungsgrad-Faktoren: (Effizienzverluste halbiert), Kosten:
f_e_pro = 1-((1 - f_e)*0.5)
f_v_pro = 1-((1 - f_v)*0.5)


nennkapazität = 40 # MWh brutto
nennkapazität_pro = 50 # Kosten:

lademinimum = 0.2 # 20%
lademaximum = 1 # 100%
anfangsbestand = 0.5 # 50%

nettokapazität = zyklus = nennkapazität * (lademaximum - lademinimum) # MWh netto
nettokapazität_pro = nennkapazität_pro * (lademaximum - lademinimum) # Kosten:

zykluskosten = 1500 # € / zyklus
mwh_zykluskosten = zykluskosten / zyklus # € / MWh

erlaubte_zyklen_pro_tag = 2
erlaubte_zyklen_pro_tag_pro = 3

a = anfangsbestand * nennkapazität # MWh Anfangs- und Endbestand
u = lademinimum * nennkapazität # MWh Untergrenze Batteriekapazität
o = lademaximum * nennkapazität # MWh Obergrenze Batteriekapazität

c = 0.5 # nennkapazität / h
c_pro = 1 # Kosten:

# Prozentuale penalty, wenn nicht Zuschlag
penalty_anteil = 1
penalty = 1 + penalty_anteil



# -> Wir haben vier Investitionsentscheidungen (f_e_pro und f_v_pro, c_pro, nennkapazität_pro und erlaubte_zyklen_pro_tag_pro)

**Vorhersagedaten**

In [6]:
import pandas as pd

prognose = pd.read_excel('Preisprognosen.xlsx')

# Erwartungswerte nach Risikoklasse und Stunde für Einkauf

preis_verkauf_h_r = {}
preis_einkauf_h_r = {}

r = 0
for h in range(1,25):
  # Berechnung von "Standard-EW" für Stunde h
  stundenprognose = prognose[prognose['Stunde'] == h]
  #Sortiere stundenprognose nach dem Wert in Spalte "Strompreis"
  stundenprognose = stundenprognose.sort_values(by='Strompreis')
  # Index zurücksetzen
  stundenprognose = stundenprognose.reset_index(drop=True)
  #print(stundenprognose)
  for i in range(0, 30*j+1):
    #iteriere außerdem j*30 über den df
    # Nehme den Durschnitt von der Spalte "Strompreis" vom Index r bis zum höchsten Index
      preis_verkauf_h_r[(h+i*24, r)] = stundenprognose.loc[r:24, 'Strompreis'].mean()
      preis_einkauf_h_r[(h+i*24, r)] = stundenprognose.loc[0:24-r, 'Strompreis'].mean()
   
    

# Für die Berechnung der Penalty möchten wir den Durchschnittspreis der umliegenden Stunden zu jeder Stunde haben
moving_average_h = {}
for h in range(1,25):
  stundenprognose = prognose[prognose['Stunde'].isin([h-1,h,h+1])]
  for i in range(0,30*j+1):
    moving_average_h[h+i*24] = stundenprognose['Strompreis'].mean()


**Entscheidungsvariablen**

In [7]:
# Für jede Risikoklasse und jede Stunde eine Mengen-Variable
e_h_r={}
v_h_r={}

# Investitionsentscheidungen
inv_c_rate = scip.addVar(vtype="B", name=f"Investition_c-Rate")
inv_nennkapazität = scip.addVar(vtype="B", name=f"Investition_Nennkapa")
inv_zyklen_pro_tag = scip.addVar(vtype="B", name=f"Investition_Zyklen_pro_Tag")
inv_wirkungsgrad = scip.addVar(vtype="B", name=f"Investition_Wirkungsgrad")

for h in H:
    #  Für jede Risikoklasse und jede Stunde eine Mengen-Variable, ob man zu dieser Stunde bei dieser Risikoklasse die entsprechende Menge einkaufen möchte
    
      e_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}_{r}")
      v_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}_{r}")


# **Zielfunktion**

In [8]:
r = 0
gewinn_kauf_verkauf = quicksum(((preis_verkauf_h_r[(h, r)] * v_h_r[(h, r)] * prob_r[r] +             \
                                 moving_average_h[h] * v_h_r[(h, r)] * (1-prob_r[r]) * 1/penalty) -       \

                                (preis_einkauf_h_r[(h, r)] * e_h_r[(h, r)] * prob_r[r] +             \
                                 moving_average_h[h] * e_h_r[(h, r)] * (1-prob_r[r]) * penalty ))        \

                               for h in H )


# Bei 3 Zyklen Investition kostet der dritte Zyklus nur 1000 mehr pro Tag (anstatt 1.500), also Skaleneffekte. Quelle: haben wir uns ausgedacht
zykluskosten = (3000 + (1000 * inv_zyklen_pro_tag)) * 30 * j

# Investitionskosten, direkt mit der Binärvariable multipliziert, sodass sie nur Anwendung finden wenn man sich für die Inv entscheidet
investitionskosten_c_rate = 50_000 * inv_c_rate
investitionskosten_nennkapazität = 50_000 *  inv_nennkapazität
investitionskosten_zyklen_pro_tag = 50_000 * inv_zyklen_pro_tag
investitionskosten_wirkungsgrad = 100_000 * inv_wirkungsgrad


scip.setObjective(gewinn_kauf_verkauf - zykluskosten - (investitionskosten_nennkapazität/12 + investitionskosten_c_rate/12 + inv_zyklen_pro_tag/12 + inv_wirkungsgrad/12), sense="maximize")

***Nebenbedingungen/ Restriktionen***

In [9]:
f_e_inv = ((f_e*(1-inv_wirkungsgrad))+(f_e_pro*inv_wirkungsgrad))
f_v_inv = ((f_v*(1-inv_wirkungsgrad))+(f_v_pro*inv_wirkungsgrad))

erlaubte_zyklen_pro_tag_inv = ((erlaubte_zyklen_pro_tag*(1-inv_zyklen_pro_tag))+(erlaubte_zyklen_pro_tag_pro*inv_zyklen_pro_tag))

nettokapazität_inv = ((nettokapazität*(1-inv_nennkapazität))+nettokapazität_pro*inv_nennkapazität)

# Ladestand zur Stunde 0 = Ladestand zur Stunde 24, also Summe Lademenge und Entlademenge gleich
scip.addCons(quicksum(((e_h_r[(h,r)] * f_e_inv) - (v_h_r[(h,r)] / f_v_inv)) for h in H) == 0, name="Anfangs- und Endbestand gleich")
print("added: Anfangs- und Endbestand gleich")

# Maximale Ladezyklen am pro Tag anhand der Einkaufsmenge (mit Faktor = Lademenge), alternativ anhand der Verkaufsmenge
scip.addCons(quicksum((e_h_r[(h,r)] * f_e_inv) for h in H) <= (erlaubte_zyklen_pro_tag_inv * nettokapazität_inv * 30 * j), name="Maximale Ladezyklen pro Tag")
print("added: Maximale Ladezyklen pro Tag")

# Mindestladestand nicht unterschritten und Höchstladestand nicht überschritten
# Mindestladestand nicht unterschritten und Höchstladestand nicht überschritten
for h in H:
    if h % 100 == 0: print(f"Stunde {h} wird hinzugefügt")
    H_t =  [n for n in range(1, h+1)]
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e_inv) - (v_h_r[(t, r)] / f_v_inv)) for t in H_t)) >= u, name=f"Mindestladestand zum Zeitpunkt t={h}")
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e_inv) - (v_h_r[(t, r)] / f_v_inv)) for t in H_t)) <= o, name=f"Maximalladestand zum Zeitpunkt t={h}")

# Lade- und Entladeleistung begrenzt (C-Rate)
for h in H:
    if h % 100 == 0: print(f"Stunde {h} wird hinzugefügt")
    scip.addCons(((e_h_r[(h, r)] * f_e_inv) + (v_h_r[(h, r)] / f_v_inv))  <= c * nennkapazität, name=f"Lade-/Entladeleistung der Stunde h={h}")

added: Anfangs- und Endbestand gleich
added: Maximale Ladezyklen pro Tag
Stunde 100 wird hinzugefügt
Stunde 200 wird hinzugefügt
Stunde 300 wird hinzugefügt
Stunde 400 wird hinzugefügt
Stunde 500 wird hinzugefügt
Stunde 600 wird hinzugefügt
Stunde 700 wird hinzugefügt
Stunde 100 wird hinzugefügt
Stunde 200 wird hinzugefügt
Stunde 300 wird hinzugefügt
Stunde 400 wird hinzugefügt
Stunde 500 wird hinzugefügt
Stunde 600 wird hinzugefügt
Stunde 700 wird hinzugefügt


**Berechnung der Lösung**

In [13]:
scip.setIntParam("display/verblevel", 5)  # Set verbosity level to 5


scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Gewinn) =', scip.getObjVal())
    for h in H:
      v_h = 0
      e_h = 0
      risikoklasse_v = None
      risikodict_v = {}
      risikoklasse_e = None
      risikodict_e = {}

      

      v_h += scip.getVal(v_h_r[(h, r)])
      e_h += scip.getVal(e_h_r[(h, r)])  
      print("EINKAUF Stunde", h, " : " , e_h)
      if risikoklasse_e != None: print("EINKAUF Risikoklasse Stunde", h, " : ", risikoklasse_e)
      #print("EINKAUF Risiko: ", risikodict_e)
      print("Verkauf Stunde", h, " : " , v_h)
      if risikoklasse_v != None: print("VERKAUF Risikoklasse Stunde", h, " : ", risikoklasse_v)
      #print("VERKAUF Risiko: ", risikodict_v)
else:
    print('Problem hat keine Lösung')

Status des Solvers: optimal 

LÖSUNG:
Zielfunktionswert (Gewinn) = 45918.23606873932
EINKAUF Stunde 1  :  0.0
Verkauf Stunde 1  :  1.1139852573157841e-13
EINKAUF Stunde 2  :  0.0
Verkauf Stunde 2  :  0.0
EINKAUF Stunde 3  :  0.0
Verkauf Stunde 3  :  0.0
EINKAUF Stunde 4  :  20.404259389147487
Verkauf Stunde 4  :  0.0
EINKAUF Stunde 5  :  1.1598478630018852e-13
Verkauf Stunde 5  :  0.0
EINKAUF Stunde 6  :  0.0
Verkauf Stunde 6  :  0.0
EINKAUF Stunde 7  :  0.0
Verkauf Stunde 7  :  11.758461537481667
EINKAUF Stunde 8  :  0.0
Verkauf Stunde 8  :  19.597435898415778
EINKAUF Stunde 9  :  0.0
Verkauf Stunde 9  :  0.0
EINKAUF Stunde 10  :  0.0
Verkauf Stunde 10  :  0.0
EINKAUF Stunde 11  :  0.0
Verkauf Stunde 11  :  0.0
EINKAUF Stunde 12  :  0.0
Verkauf Stunde 12  :  0.0
EINKAUF Stunde 13  :  0.0
Verkauf Stunde 13  :  0.0
EINKAUF Stunde 14  :  12.242555633488491
Verkauf Stunde 14  :  0.0
EINKAUF Stunde 15  :  20.404259389147487
Verkauf Stunde 15  :  0.0
EINKAUF Stunde 16  :  0.0
Verkauf Stunde

In [14]:
print(scip.getVal(inv_c_rate))
print(scip.getVal(inv_nennkapazität))
print(scip.getVal(inv_zyklen_pro_tag))
print(scip.getVal(inv_wirkungsgrad))


-0.0
0.0
0.0
1.0000000000000058


In [15]:
sum_v = 0
sum_e = 0
for h in H: 
    sum_v += scip.getVal(v_h_r[(h, r)])
    sum_e += scip.getVal(e_h_r[(h, r)])
    
print(sum_v)
print(sum_e)

1881.3538461538499
1958.808901358159
